<a href="https://colab.research.google.com/github/MengOonLee/AccountReceivable/blob/main/Workflow/InvoicePayment/Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Invoice Payment Forecasting

Accounts Receivables (AR) is an important aspect of businesses, it is a credit extended by the company to the client that require payments due within a time period. With B2B transactions increasing in volume and complexity, poor management of AR can lead to to unnecessary cash flow problems.

Invoice payment forecasting can be used to optimize collection strategies that allows businesses to predict and plan for future customer payments. By analyzing customer payment history, businesses can forecast their future accounts receivable balance and plan accordingly. By identifying invoices that are likely to be paid late, customers who are likely to have large amounts of overdue receivables, which in turn can lead to better visibility into future cash flow.

The invoices dataset contains the past payment information and behaviour of various buyers [here](https://www.kaggle.com/datasets/hhenry/finance-factoring-ibm-late-payment-histories).



In [ ]:
%%bash
pip install --no-cache-dir -qU \
    datasets bitsandbytes peft
pip check

In [20]:
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [105]:
import os
import pandas as pd

input_dir = '/content/drive/MyDrive/Data'
input_invoice = 'WA_Fn-UseC_-Accounts-Receivable.csv'
df_invoice = pd.read_csv(os.path.join(input_dir, input_invoice))
df_invoice.info()
df_invoice.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2466 entries, 0 to 2465
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   countryCode    2466 non-null   int64  
 1   customerID     2466 non-null   object 
 2   PaperlessDate  2466 non-null   object 
 3   invoiceNumber  2466 non-null   int64  
 4   InvoiceDate    2466 non-null   object 
 5   DueDate        2466 non-null   object 
 6   InvoiceAmount  2466 non-null   float64
 7   Disputed       2466 non-null   object 
 8   SettledDate    2466 non-null   object 
 9   PaperlessBill  2466 non-null   object 
 10  DaysToSettle   2466 non-null   int64  
 11  DaysLate       2466 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 231.3+ KB


,countryCode,customerID,PaperlessDate,invoiceNumber,InvoiceDate,DueDate,InvoiceAmount,Disputed,SettledDate,PaperlessBill,DaysToSettle,DaysLate
0,391,0379-NEVHP,4/6/2013,611365,1/2/2013,2/1/2013,55.94,No,1/15/2013,Paper,13,0
1,406,8976-AMJEO,3/3/2012,7900770,1/26/2013,2/25/2013,61.74,Yes,3/3/2013,Electronic,36,6
2,391,2820-XGXSB,1/26/2012,9231909,7/3/2013,8/2/2013,65.88,No,7/8/2013,Electronic,5,0
3,406,9322-YCTQO,4/6/2012,9888306,2/10/2013,3/12/2013,105.92,No,3/17/2013,Electronic,35,5
4,818,6627-ELFBK,11/26/2012,15752855,10/25/2012,11/24/2012,72.27,Yes,11/28/2012,Paper,34,4


In [106]:
import pandas as pd

df_invoice.drop(columns=['countryCode', 'PaperlessDate', 'invoiceNumber',
    'DueDate', 'Disputed', 'SettledDate', 'PaperlessBill', 'DaysLate'],
    inplace=True)

df_invoice['InvoiceDate'] = pd.to_datetime(df_invoice['InvoiceDate'],
    format='%m/%d/%Y').dt.date

df_invoice.sort_values(by=['customerID', 'InvoiceDate'], inplace=True)

df_invoice.head()

,customerID,InvoiceDate,InvoiceAmount,DaysToSettle
991,0187-ERLSR,2012-03-29,62.68,27
2345,0187-ERLSR,2012-05-15,77.19,13
2401,0187-ERLSR,2012-05-21,51.65,14
1791,0187-ERLSR,2012-06-16,64.47,18
445,0187-ERLSR,2012-09-05,84.57,9


In [114]:
import numpy as np

print("Total number of customer:", len(df_invoice['customerID'].unique()))

df_sample = df_invoice.groupby('customerID')\
    .agg(list)\
    .reset_index()

df_sample.info()
df_sample.head()

Total number of customer: 100
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     100 non-null    object
 1   InvoiceDate    100 non-null    object
 2   InvoiceAmount  100 non-null    object
 3   DaysToSettle   100 non-null    object
dtypes: object(4)
memory usage: 3.2+ KB


,customerID,InvoiceDate,InvoiceAmount,DaysToSettle
0,0187-ERLSR,"[2012-03-29, 2012-05-15, 2012-05-21, 2012-06-1...","[62.68, 77.19, 51.65, 64.47, 84.57, 65.26, 59....","[27, 13, 14, 18, 9, 19, 13, 14, 7, 19, 5, 8, 8..."
1,0379-NEVHP,"[2012-02-12, 2012-03-01, 2012-03-20, 2012-07-0...","[28.21, 48.65, 103.64, 42.25, 69.55, 72.97, 50...","[16, 47, 19, 16, 22, 18, 13, 22, 16, 13, 21, 1..."
2,0465-DTULQ,"[2012-01-04, 2012-01-09, 2012-01-30, 2012-04-1...","[55.91, 40.22, 59.34, 60.35, 43.62, 43.09, 39....","[31, 35, 51, 23, 45, 24, 48, 35, 30, 45, 50, 3..."
3,0625-TNJFG,"[2012-02-24, 2012-03-06, 2012-03-20, 2012-06-1...","[62.39, 41.89, 27.08, 48.08, 64.44, 42.18, 46....","[30, 22, 22, 16, 30, 20, 24, 23, 24, 32, 24, 3..."
4,0688-XNJRO,"[2012-01-12, 2012-01-18, 2012-02-08, 2012-02-1...","[64.19, 18.03, 68.28, 27.22, 35.3, 49.62, 45.7...","[43, 64, 46, 33, 49, 48, 51, 41, 57, 40, 45, 4..."


In [119]:
f_instruction = lambda x: f"How many days to settle invoice amount {x[-1]}?"
df_sample['instruction'] = df_sample['InvoiceAmount'].apply(f_instruction)

df_sample

,customerID,InvoiceDate,InvoiceAmount,DaysToSettle,instruction
0,0187-ERLSR,"[2012-03-29, 2012-05-15, 2012-05-21, 2012-06-1...","[62.68, 77.19, 51.65, 64.47, 84.57, 65.26, 59....","[27, 13, 14, 18, 9, 19, 13, 14, 7, 19, 5, 8, 8...",How many days to settle invoice amount 65.57?
1,0379-NEVHP,"[2012-02-12, 2012-03-01, 2012-03-20, 2012-07-0...","[28.21, 48.65, 103.64, 42.25, 69.55, 72.97, 50...","[16, 47, 19, 16, 22, 18, 13, 22, 16, 13, 21, 1...",How many days to settle invoice amount 59.56?
2,0465-DTULQ,"[2012-01-04, 2012-01-09, 2012-01-30, 2012-04-1...","[55.91, 40.22, 59.34, 60.35, 43.62, 43.09, 39....","[31, 35, 51, 23, 45, 24, 48, 35, 30, 45, 50, 3...",How many days to settle invoice amount 72.36?
3,0625-TNJFG,"[2012-02-24, 2012-03-06, 2012-03-20, 2012-06-1...","[62.39, 41.89, 27.08, 48.08, 64.44, 42.18, 46....","[30, 22, 22, 16, 30, 20, 24, 23, 24, 32, 24, 3...",How many days to settle invoice amount 92.48?
4,0688-XNJRO,"[2012-01-12, 2012-01-18, 2012-02-08, 2012-02-1...","[64.19, 18.03, 68.28, 27.22, 35.3, 49.62, 45.7...","[43, 64, 46, 33, 49, 48, 51, 41, 57, 40, 45, 4...",How many days to settle invoice amount 25.19?
...,...,...,...,...,...
95,9758-AIEIK,"[2012-01-31, 2012-02-13, 2012-03-11, 2012-03-2...","[81.13, 55.42, 61.19, 37.39, 56.25, 52.88, 51....","[20, 19, 41, 28, 38, 19, 17, 18, 11, 35, 20, 1...",How many days to settle invoice amount 43.82?
96,9771-QTLGZ,"[2012-01-20, 2012-05-16, 2012-05-19, 2012-05-2...","[56.51, 32.88, 68.89, 55.54, 67.4, 29.85, 51.1...","[26, 15, 28, 27, 13, 21, 24, 19, 19, 19, 16, 1...",How many days to settle invoice amount 50.83?
97,9841-XLGBV,"[2012-01-09, 2012-03-16, 2012-03-31, 2012-05-0...","[41.99, 46.25, 44.43, 62.1, 38.84, 53.81, 47.5...","[28, 39, 24, 28, 28, 28, 28, 30, 24, 24, 22, 2...",How many days to settle invoice amount 65.96?
98,9883-SDWFS,"[2012-01-07, 2012-03-02, 2012-03-29, 2012-04-1...","[46.33, 26.37, 14.94, 20.99, 28.77, 41.25, 19....","[34, 28, 45, 40, 37, 43, 24, 43, 26, 46, 37, 3...",How many days to settle invoice amount 32.02?


In [99]:
def create_prompt_formats(data):

    # Initialize static strings for the prompt template
    INTRO_BLURB = """
        Below is an instruction that describes a task.
        Write a response that appropriately completes the request.
    """
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "Input:"
    RESPONSE_KEY = "### Response:"
    END_KEY = "### End"

    # Combine a prompt with the static strings
    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{['']}"

datetime.date(2013, 3, 16)

In [ ]:
# Import libraries
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import argparse
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset

# Reproducibility
seed = 42
set_seed(seed)

/Work/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-13 12:15:11.697825: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-13 12:15:11.697885: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-13 12:15:11.697943: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Preprocessing

List of the fields in dataset are as follows:
* Customer: Customer code of the account
* Country: customer country
* Region:
* Account Owner Name
* Last Payment Date
* Contact Date
* Collection Specialist
* Projected Overdue Amount
* Current Overdue Amount
* Collected
* Forecast
* Forecast Status

In [ ]:
# Load the large language model file
from llama_cpp import Llama
LLM = Llama(model_path="./llama-2-7b-chat.ggmlv3.q8_0.bin")

prompt = "Q: Why are Jupyter notebooks difficult to maintain? A:"

output = LLM(prompt, max_tokens=0)

print(output["choices"][0]["text"])

In [ ]:
import xgboost as xgb